In [129]:
import pandas as pd
import matplotlib.pyplot as plt
df_GDP=pd.read_csv("GDP.csv")
df_INF=pd.read_csv("INF.csv")
df_UNEMP=pd.read_csv("UNEMP.csv")

In [ ]:
# First of all, as i want to perform a 30 year analysis on the 3 datasets (30 years is the best period talking about consistency in these dataset), i need to drop and clean 
dfs=[df_GDP, df_INF, df_UNEMP]

for df in dfs:
    df.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 68'], inplace=True)
    df.set_index('Country Name', inplace=True)

In [140]:
dfs2 = [ df_GDP, df_INF, df_UNEMP]
nn_dfs=[]
for df in dfs2:
    nn_dfs.append(df.loc[:, df.columns.astype(int) > 1993])

df_GDP, df_INF, df_UNEMP = nn_dfs

In [132]:
#Good! Now that we have the same raws and columns for every dataframe, we need to check what is still NaN and drop
df_GDP.isnull().sum()
df_GDP=df_GDP.dropna().copy()
df_INF=df_INF.dropna().copy()

In [133]:
#Now as we want to work on the same countries in every Df, we need to filter them by index
common_index = df_GDP.index.intersection(df_UNEMP.index).intersection(df_INF.index)
filtered_dfs = []

for df in dfs:
    filtered_dfs.append(df.loc[common_index])

df_GDP_filtered, df_INF_filtered, df_UNEMP_filtered = filtered_dfs 

In [134]:
#I want readable dataframes: convert to billions GDP and only 2 decimals. Way better!

df_GDP_filtered.columns = df_GDP_filtered.columns.astype(int)  # Converti i nomi delle colonne in int
df_GDP_filtered.loc[:, 1994:2023] = df_GDP_filtered.loc[:, 1994:2023] / 1_000_000_000
df_GDP_filtered = df_GDP_filtered.round(2)
df_INF_filtered = df_INF_filtered.round(2)
df_UNEMP_filtered = df_UNEMP_filtered.round(2)

In [ ]:
#Now that we achieved a readable dataframe, the best way to work is convert from wide to long format using melt, and the merge to have a all-in-one dataframe.

df_inflation_long = df_INF_filtered.reset_index().melt(id_vars=['Country Name'], var_name='Year', value_name='Inflation')
df_UNEMP_long = df_UNEMP_filtered.reset_index().melt(id_vars=['Country Name'], var_name='Year', value_name='Unemployment')
df_GDP_long = df_GDP_filtered.reset_index().melt(id_vars=['Country Name'], var_name='Year', value_name='GDP')

for df in [df_GDP_long, df_inflation_long, df_UNEMP_long]:
    df["Year"] = df["Year"].astype(int)

df_merged = df_GDP_long.merge(df_inflation_long, on=["Country Name", "Year"], how="inner") \
                  .merge(df_UNEMP_long, on=["Country Name", "Year"], how="inner")
#Way better! This will be good for Tableau!
df_merged.set_index('Country Name')
df_merged.to_csv('Macroeconomics_dataset.csv')